In [1]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import re

In [2]:
column_names = {'Emiasid': 'emiasid', 'Emiasid ': 'emiasid' , 'emiasid ': 'emiasid', 'EMIAS_ID': 'emiasid', 'МО': 'mo', 
                'МУ': 'mu', 'Логин оператора': 'login', 'Дата и время звонка': 'call_date', 'Результат звонка': 'call_result',
                'Результат звонка ': 'call_result', 'Разговор состоялся': 'talk_result', 'Комменатрий': 'comment', 
                'Дата и время переноса': 'next_call_date', 'Контактный телефон': 'phone_number', 'ФИО врача': 'doctor_fio', 
                'ФИО помощника врача': 'assistant_fio', 'Логин помощника врача': 'login', 'Зал №': 'hall_number',
                'Логин naumen': 'login', 'МО закрепления': 'mo', 'Отсутствует на рабочем месте': 'absence'}

In [3]:
today = pd.Timestamp(dt.datetime.today().date())
today_ymd = today.strftime('%Y%m%d')
today_y_m_d = today.strftime('%Y_%m_%d')

yesterday = today - dt.timedelta(days=1)
yesterday_ymd = yesterday.strftime('%Y%m%d')
yesterday_y_m_d = yesterday.strftime('%Y-%m-%d')

tomorrow = today + dt.timedelta(days=1)
tomorrow_ymd = tomorrow.strftime('%Y%m%d')
tomorrow_y_m_d = tomorrow.strftime('%Y-%m-%d')

print(today_y_m_d)

2022_09_15


In [4]:
# путь файла с текущими статусами пациентов
patients_current_statuses_path = \
r'\Users\murad_satabaev\Documents\GitHub\final_framework\merged_statuses'

patients_current_statuses_path = patients_current_statuses_path.split('\\')
patients_current_statuses_data = os.path.join('C:\\', r'\\', *patients_current_statuses_path,
                                             f'total_patients_current_statuses_merged_with_logins_{today_y_m_d}.xlsx')
print(patients_current_statuses_data)

C:\\Users\murad_satabaev\Documents\GitHub\final_framework\merged_statuses\total_patients_current_statuses_merged_with_logins_2022_09_15.xlsx


In [5]:
# путь файла с сегодняшними 
emias_data_path = r'\\t999\Сетевой диск\pdn\data\emias_data\xlsx'
emias_data_path = emias_data_path.split('\\')
emias_data_path = os.path.join(r'\\', *emias_data_path)
print(emias_data_path)

\\t999\Сетевой диск\pdn\data\emias_data\xlsx


In [6]:
today_emias_data_path = [os.path.join(emias_data_path, filename) for filename in os.listdir(emias_data_path) if '4059' 
                         in filename and '~' not in filename and today.strftime('%Y-%m-%d') in filename][0]
print(today_emias_data_path)

\\t999\Сетевой диск\pdn\data\emias_data\xlsx\DAT_4059_выгрузка_списков_на_обзвон_помощниками_2022-09-15.xlsx


In [7]:
# чтение всех файлов по отдельности, чтобы не тратить время на повторное чтение файла, который уже прочитан

In [8]:
today_emias_data_reading = pd.read_excel(today_emias_data_path)
today_emias_data_reading.shape[0]

97331

In [9]:
patients_current_statuses_reading = pd.read_excel(patients_current_statuses_data)
patients_current_statuses_reading.shape[0]

109888

In [10]:
import gspread
from gspread_dataframe import get_as_dataframe
gc = gspread.service_account(filename='google_sheets_credentials.json') 
url = 'https://docs.google.com/spreadsheets/d/1TXjbk9tRDMCjEyyiuK5OIgT8xwlHQ2V1PX2zaVnxPXY/edit#gid=0'
sh = gc.open_by_url(url)
ws = sh.get_worksheet(0)
def search_row(ws, desired_value):
    row_number = 0
    for row in ws.get_all_values():
        row_number += 1
        for value in row:
            if value == desired_value:
                return row_number
            
skiprows = search_row(ws, 'ФИО помощника врача') - 1

assistants_presence_reading = get_as_dataframe(ws, evaluate_formulas=True, skiprows=skiprows)
assistants_presence_reading = assistants_presence_reading.loc[~(assistants_presence_reading['Зал №'] == 'Зал №'), 
    ['Зал №', 'МО закрепления', 'ФИО помощника врача', 'Логин naumen', 'Отсутствует на рабочем месте']] \
        .dropna(how='all', axis=0)


print(dt.datetime.now())
print(assistants_presence_reading.shape[0])

assistants_presence = assistants_presence_reading.rename(columns=column_names).copy()

2022-09-15 20:05:13.063688
217


In [11]:
patients_current_statuses = patients_current_statuses_reading.rename(columns=column_names).copy()

In [12]:
today_emias_data = today_emias_data_reading.rename(columns=column_names)

In [13]:
patients_current_statuses = patients_current_statuses_reading.rename(columns=column_names).copy()
patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')
patients_current_statuses['next_call_date'] = patients_current_statuses['next_call_date'] \
.apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
patients_current_statuses['last_call_date'] = patients_current_statuses['last_call_date'] \
.apply(lambda x: pd.to_datetime(x, dayfirst=True).date())

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_5084\1251941699.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')


In [ ]:
# Переведем все типы данных емиасайди, телефоны и прочие числовые данные в строки

In [14]:
today_emias_data['emiasid'] = today_emias_data['emiasid'].astype(str).str.replace('\.0', '')
today_emias_data['phone'] = today_emias_data['phone'].astype(str).str.replace('\.0', '')

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_5084\3971369908.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  today_emias_data['emiasid'] = today_emias_data['emiasid'].astype(str).str.replace('\.0', '')
C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_5084\3971369908.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  today_emias_data['phone'] = today_emias_data['phone'].astype(str).str.replace('\.0', '')


In [15]:
# путь папки с раскреплением
assistant_to_doctor_distribution_path = \
r'\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path'
assistant_to_doctor_distribution_path = \
assistant_to_doctor_distribution_path.split('\\')
assistant_to_doctor_distribution_path = \
os.path.join(r'\\', *assistant_to_doctor_distribution_path)
print(assistant_to_doctor_distribution_path)

\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path


In [16]:
september_assistant_to_doctor_distribution_path = [os.path.join(assistant_to_doctor_distribution_path, filename) for filename 
                                         in os.listdir(assistant_to_doctor_distribution_path) if '~' not in filename
                                        and 'september' in filename]
september_assistant_to_doctor_distribution_path.sort(key=lambda x: os.path.getctime(x))
september_assistant_to_doctor_distribution_path = september_assistant_to_doctor_distribution_path[-1]
print(september_assistant_to_doctor_distribution_path)

\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path\september_patient_to_assistant_distribution.xlsx


In [17]:
september_assistant_to_doctor_distribution = pd.read_excel(september_assistant_to_doctor_distribution_path, 
                                    usecols=['login', 'emiasid', 'mo', 'doctor_fio'])[['emiasid', 'login', 'mo', 'doctor_fio']]

In [18]:
def find_phone_number(text):
    """Функция ищет номера телефонов в строке, фильтрует дефектные и возвращает один с приоритетом на мобильный"""
    import re
    defect_pattern = re.compile(r"""7[^94]\d{9}|0{7,}|1{7,}|2{7,}|3{7,}|4{7,}|5{7,}|6{7,}|7{7,}|8{7,}|9{7,}""")
    text = str(text)
    text = ['7' + re.sub(r'\D', '', x)[-10:] for x in re.split(r'[,;]', text)]
    text = list(filter(lambda x: len(x) == 11, text))
    text = list(filter(lambda x: False if re.search(defect_pattern, x) else True, text))
    text = sorted(text, reverse=True)
    text = ','.join(text)
    text = re.search(r'(9\d{9}|4\d{9})', text)
    if text:
        text = text.group()
    return text

today_emias_data['phone_1'] = today_emias_data['phone'].apply(find_phone_number)
today_emias_data[today_emias_data['phone_1'].isna()].shape[0]

877

In [19]:
today_emias_data.shape[0]

97331

In [20]:
patients_current_statuses.shape[0]

109888

In [21]:
exception = pd.read_excel('exception.xlsx')

In [22]:
exception = exception.rename(columns=column_names)

In [262]:
patients_current_statuses.problem.unique()

array([nan, 'Нет в сегодняшней выгрузке', 'Нет прикрепления к врачу',
       'Смена МО на другую нашу', 'Нет прикрепления к МО',
       'Отсутствует номер', 'Смена МО'], dtype=object)

In [267]:
actual_patients = \
patients_current_statuses[
    (~(patients_current_statuses.problem.isin({
    'Нет в сегодняшней выгрузке', 'Нет прикрепления к МО', 'Отсутствует номер', 'Смена МО'
    })))
    &
    (~(patients_current_statuses.emiasid.isin(exception.emiasid)))
]

In [268]:
patients_current_statuses.shape[0]

109888

In [269]:
actual_patients.shape[0]

108850

In [270]:
actual_patients.status.value_counts()

Не дозвонились                    41345
Перенос звонка на другой день     31041
Будет в Москве в другом месяце    14660
Разговор состоялся                 6790
Ни разу не звонили                 5364
Запишется самостоятельно           4422
Перезвонить сегодня                1744
Отказ                               901
Пациент отрицает диагноз            872
Иная МО                             479
Госпитализирован                    361
Живет не в Москве                   282
Патронаж                            205
Некорректный номер                  114
Не нравится врач                     95
Отсутствует номер                    74
Другой врач в этой МО                32
Не нравится МО                       21
Cнят с ДН                            13
Умер                                  8
Смена МО                              5
Name: status, dtype: int64

In [271]:
if assistants_presence[assistants_presence.absence.isna()].login.nunique() + \
assistants_presence[assistants_presence.absence == 1].login.nunique() == assistants_presence.login.nunique():
    print('\nВ сумме количество отсуствующих и присутствующих равны общему количеству помощников\n')
else:
    print(5 * '\nПРОВЕРЬТЕ ТАБЛИЦУ ОТСУТСТВУЮЩИХ И ПРИСУТСТВУЮЩИХ\n!!!\n')


В сумме количество отсуствующих и присутствующих равны общему количеству помощников



In [272]:
present_assistants = assistants_presence[assistants_presence.absence.isna()]
print(present_assistants.shape[0])
absent_assistants = assistants_presence[~assistants_presence.absence.isna()]
print(absent_assistants.shape[0])

168
49


In [273]:
doctors_list = set(september_assistant_to_doctor_distribution.doctor_fio.unique())
doctors_list.add(np.NaN)

In [488]:
first_stage_patients = \
actual_patients[actual_patients.status == 'Ни разу не звонили']

first_stage_patients['priority'] = 2
first_stage_patients.shape[0]

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_5084\3421971818.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_stage_patients['priority'] = 2


5364

In [489]:
call_back_on_date_patients = \
actual_patients[
    (actual_patients.next_call_date.isin({
        dt.date(2022, 9, 14),
        dt.date(2022, 9, 15),
        dt.date(2022, 9, 16)
    }))
]

call_back_on_date_patients['priority'] = 1
call_back_on_date_patients.shape[0]

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_5084\1933833767.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  call_back_on_date_patients['priority'] = 1


5918

In [490]:
actual_patients[
    (actual_patients.month == 'august')
    &
    (actual_patients.last_call_date < dt.date(2022, 9, 2))
]

,emiasid,operator_login,status,old_comment,next_call_date,mo,doctor_fio,phone_number,calls_amount,last_call_date,last_call_result,last_talk_result,problem,month,group,operator_absence
3,18474691,NaN,Будет в Москве в другом месяце,"Не в Москве, сентябрь",NaT,ГП 191,Зуева Наталья Александровна,9.032505e+09,1.0,2022-08-02,Разговор состоялся,Будет в Москве в другом месяце,NaN,august,NaN,NaN
6,22571949,NaN,Будет в Москве в другом месяце,"не в Москве , сентябрь",NaT,ГП 2,Паруш Станислав Николаевич,9.165472e+09,1.0,2022-08-02,Разговор состоялся,Будет в Москве в другом месяце,NaN,august,NaN,NaN
10,1886196716,NaN,Будет в Москве в другом месяце,"Не в Москве, сентябрь",NaT,ГП 6,Шитенкова Дария Витальевна,9.169564e+09,1.0,2022-08-02,Разговор состоялся,Будет в Москве в другом месяце,NaN,august,NaN,NaN
13,17096082,Помощник 5,Будет в Москве в другом месяце,"не в Москве, сентябрь",NaT,ГП 36,Мироненко Александра Александровна,9.154914e+09,1.0,2022-08-02,Разговор состоялся,Будет в Москве в другом месяце,NaN,august,NaN,1.0
15,17643298,pomoshnik_pdn20,Будет в Москве в другом месяце,"Не в Москве, сентябрь",NaT,ГП 191,Баранова Яна Юрьевна,9.067388e+09,1.0,2022-08-02,Разговор состоялся,Будет в Москве в другом месяце,NaN,august,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12572,16804776,pomoshnik_pdn2,Запишется самостоятельно,"Перезвонить , самозапись эхо моча и терапевт",2022-09-06,КДЦ 4,Бородай Евгений Борисович,9.851079e+09,3.0,2022-08-31,Перенос звонка на другой день,NaN,NaN,august,NaN,NaN
12573,17181146,pomoshnik_pdn75,Запишется самостоятельно,"перезвонить, запишутся самостоятельно к терапевту",2022-09-09,ГП 134,Колосветова Ирина Игоревна,9.060762e+09,3.0,2022-08-29,Перенос звонка на другой день,NaN,NaN,august,NaN,1.0
12576,17642637,NaN,Запишется самостоятельно,NaN,NaT,ГП 191,Костяная Мария Олеговна,9.096308e+09,3.0,2022-08-31,Нет ответа,NaN,NaN,august,NaN,NaN
12577,17707347,pomoshnik_pdn2,Запишется самостоятельно,Не хочет говорить,2022-09-06,КДЦ 4,Бородай Евгений Борисович,9.166246e+09,2.0,2022-08-24,Перенос звонка на другой день,NaN,NaN,august,NaN,NaN


In [491]:
august_patients = \
actual_patients[
    (actual_patients.month == 'august')
    &
    (actual_patients.last_call_result != 'Перенос звонка на другой день')
    &
    (actual_patients.last_call_date < dt.date(2022, 9, 2))
]

august_patients['priority'] = 3
august_patients.shape[0]

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_5084\1560472559.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  august_patients['priority'] = 3


2364

In [492]:
august_patients.last_call_date.unique()[-1]

datetime.date(2022, 9, 1)

In [493]:
tomorrow_record_patients_set = set()
for column in today_emias_data.columns[22:98:2]:
    tomorrow_record_patients_set = \
    tomorrow_record_patients_set | set(today_emias_data[today_emias_data[f'{column}'] == tomorrow_y_m_d]\
                                   .emiasid.astype(str).unique())

print(len(tomorrow_record_patients_set))

0


In [494]:
tomorrow_record_patients = patients_current_statuses[(patients_current_statuses.emiasid.isin(tomorrow_record_patients_set))
& (~(patients_current_statuses.status.isin({'Ни разу не звонили'}))) & 
(patients_current_statuses.last_call_result != 'Перенос звонка на другой день')]
print(tomorrow_record_patients.shape[0])

0


In [495]:
tomorrow_record_patients['priority'] = 4

In [496]:
tomorrow_record_patients.status.value_counts()

Series([], Name: status, dtype: int64)

In [497]:
tomorrow_record_patients['status'] = 'Напомнить'

In [498]:
second_stage_patients = \
actual_patients[
    (actual_patients.status.isin({'Разговор состоялся', 'Не дозвонились', 'Другой врач в этой МО'
                                'Запишется самостоятельно', 'Не нравится МО', 'Не нравится врач'}))
    &
    (~(actual_patients.emiasid.isin(call_back_on_date_patients.emiasid)))
    &
    (~(actual_patients.last_call_date.isin(
        {
        dt.date(2022, 9, 13),
        dt.date(2022, 9, 14),
        dt.date(2022, 9, 15)
        }
    )))
]

second_stage_patients['priority'] = 5

second_stage_patients.shape[0]

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_5084\1793606296.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_stage_patients['priority'] = 5


27686

In [403]:
final_actual_patients_for_task_ = pd.concat([
    call_back_on_date_patients,
    first_stage_patients,
#     august_patients,
#     second_stage_patients,
#     tomorrow_record_patients
]) \
.sort_values(['priority']).reset_index(drop=True)

final_actual_patients_for_task_.shape[0]

9809

In [404]:
final_actual_patients_for_task = final_actual_patients_for_task_.drop_duplicates(subset=['phone_number'], keep='first')
final_actual_patients_for_task.shape[0]

9775

In [405]:
final_actual_patients_for_task = final_actual_patients_for_task.drop_duplicates(subset=['emiasid'], keep='first')
final_actual_patients_for_task.shape[0]

9775

In [406]:
final_actual_patients_for_task.head(3)

,emiasid,operator_login,status,old_comment,next_call_date,mo,doctor_fio,phone_number,calls_amount,last_call_date,last_call_result,last_talk_result,problem,month,group,operator_absence,priority
0,1126204316,Помощник 7,Ни разу не звонили,NaN,NaT,ГП 166,Ризвонов Максуджон Мавлонович,9.857766e+09,NaN,NaT,NaN,NaN,NaN,september,11.0,1.0,1
1,17444838,pomoshnik_pdn50,Ни разу не звонили,NaN,NaT,ГП 166,Житлова Юлия Сергеевна,9.154611e+09,NaN,NaT,NaN,NaN,NaN,september,11.0,NaN,1
2,17445496,Помощник 8,Ни разу не звонили,NaN,NaT,ГП 166,Ковыршина Екатерина Павловна,9.166623e+09,NaN,NaT,NaN,NaN,NaN,september,13.0,1.0,1


In [499]:
final_actual_patients_for_task = \
final_actual_patients_for_task[['emiasid', 'status', 'old_comment', 'operator_login', 'mo', 'doctor_fio', 'priority', 'calls_amount',
                                'last_call_date', 'month']]

In [500]:
final_actual_patients_for_task['emiasid'] = final_actual_patients_for_task['emiasid'].astype(str).str.replace('\.0', '')

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_5084\2998909243.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  final_actual_patients_for_task['emiasid'] = final_actual_patients_for_task['emiasid'].astype(str).str.replace('\.0', '')


In [501]:
initial_assistant_to_patient_distribution_in_today_task = \
final_actual_patients_for_task.copy()

In [502]:
initial_assistant_to_patient_distribution_in_today_task.old_comment.nunique()

2066

In [503]:
initial_assistant_to_patient_distribution_in_today_task.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(
    ['mo', 'emiasid']).head(20)

emiasid
mo     operator_login           
ГП 107 pomoshnik_pdn130       11
       pomoshnik_pdn109       12
       pomoshnik_pdn43        22
       pomoshnik_pdn129       37
       pomoshnik_pdn185       39
       pomoshnik_pdn148       44
ГП 109 pomoshnik_pdn95         1
       pomoshnik_pdn155       10
       pomoshnik_pdn168       30
       pomoshnik_pdn134       34
ГП 11  Помощник 7              1
       pomoshnik_pdn32        13
       pomoshnik_pdn184       17
       pomoshnik_pdn147       62
       pomoshnik_pdn146       80
ГП 134 pomoshnik_pdn7         13
       pomoshnik_pdn75        24
       pomoshnik_pdn89        35
       pomoshnik_pdn93        54
       pomoshnik_pdn63        59

In [412]:
set(present_assistants.login.unique()) - \
(set(initial_assistant_to_patient_distribution_in_today_task.operator_login.unique()) - {np.nan})

{'pomoshnik_pdn131', 'pomoshnik_pdn82'}

In [413]:
# initial_assistant_to_patient_distribution_in_today_task.to_excel('initial_assistant_to_patient_distribution_in_today_task.xlsx')

In [414]:
task_amount = 300

In [415]:
%%time

initial_task_for_today = pd.DataFrame()
for operator_login in set(initial_assistant_to_patient_distribution_in_today_task.operator_login):
    initial_assistant_to_patient_distribution_in_today_task_for_operator_login = \
    initial_assistant_to_patient_distribution_in_today_task[initial_assistant_to_patient_distribution_in_today_task \
                                                            .operator_login
         == operator_login].sort_values(by=['priority', 'last_call_date', 'calls_amount'])
    call_back_number_for_operator_login = initial_assistant_to_patient_distribution_in_today_task_for_operator_login[
        initial_assistant_to_patient_distribution_in_today_task_for_operator_login.status.isin({
            'Перезвонить сегодня', 'Перенос звонка на другой день'})].shape[0]

    initial_task_for_today = pd.concat(
        [initial_task_for_today, 

         
         initial_assistant_to_patient_distribution_in_today_task[
             (initial_assistant_to_patient_distribution_in_today_task.operator_login==operator_login)
         ].head(task_amount),
        
       
        ]).reset_index(drop=True)

initial_task_for_today.shape[0]

CPU times: total: 1.3 s
Wall time: 1.28 s


9214

In [416]:
initial_task_for_today \
.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'])\
.sort_values(by='emiasid', ascending=True).head(10)

emiasid
mo     operator_login           
ГП 19  pomoshnik_pdn246        1
ГП 69  pomoshnik_pdn264        1
ГП 218 pomoshnik_pdn134        1
ГП 212 Помощник 5              1
ГП 9   pomoshnik_pdn10         1
       Помощник 12             1
ГП 209 pomoshnik_pdn32         1
ГП 220 pomoshnik_pdn10         1
ГП 64  pomoshnik_pdn158        1
       pomoshnik_pdn152        1

In [417]:
initial_task_for_today \
.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'])\
.sort_values(by='emiasid', ascending=False).head(10)

,,emiasid
mo,operator_login,
ГП 214,pomoshnik_pdn156,300
ГП 23,pomoshnik_pdn14,300
ГП 166,Помощник 8,300
ГП 191,pomoshnik_pdn13,300
ГП 166,Помощник 7,299
ГП 2,pomoshnik_pdn83,295
ГП 23,pomoshnik_pdn10,294
ГП 191,pomoshnik_pdn52,285
ГП 45,pomoshnik_pdn307,249


In [418]:
rename_mo = \
{'ГБУЗ Г МОСКВЫ ГП № 170 ДЗМ': 'ГП 170', ' ГП 6 ДЗМ': 'ГП 6', 'ГБУЗ КДЦ 6 ДЗМ':'КДЦ 6', 
'ГБУЗ "ГП № 134 ДЗМ"': 'ГП 134', 'КДЦ 4': 'КДЦ 4', 'ГБУЗ ДКЦ 1 ДЗМ': 'ДКЦ 1', 'ГБУЗ ГП 23 ДЗМ': 'ГП 23',
'ГБУЗ ГП 109 ДЗМ': 'ГП 109', 'ГБУЗ ГП 11 ДЗМ': 'ГП 11', 'ГБУЗ ГП 64 ДЗМ': 'ГП 64', 'ГБУЗ ГП 191 ДЗМ': 'ГП 191', 
'ГБУЗ ГП 220 ДЗМ': 'ГП 220', 'ГБУЗ "ГП № 218 ДЗМ"': 'ГП 218',  'ГБУЗ ГП 107 ДЗМ': 'ГП 107', 
'ГБУЗ "ГП № 22 ДЗМ"': 'ГП 22', 'ГБУЗ ДЦ 5 ДЗМ': 'ДЦ 5', 'ГБУЗ ГП 210 ДЗМ': 'ГП 210', 'ГБУЗ ГП 214 ДЗМ': 'ГП 214', 
'ГБУЗ ГП 2 ДЗМ': 'ГП 2', 'ГБУЗ КДП 121 ДЗМ': 'КДП 121', 'ГБУЗ ГП 66 ДЗМ': 'ГП 66',  'ГБУЗ ДЦ 3 ДЗМ ': 'ДЦ 3',
'ГБУЗ ГП 36 ДЗМ': 'ГП 36',  'ГБУЗ ГП 62 ДЗМ ': 'ГП 62', 
'ГБУЗ "ГП № 52 ДЗМ"': 'ГП 52', 'ГБУЗ "ГП 67 ДЗМ"': 'ГП 67', 'ГБУЗ ГП 68 ДЗМ': 'ГП 68', 'ГБУЗ ГП 175 ДЗМ': 'ГП 175',
'ГБУЗ ГП № 209 ДЗМ': 'ГП 209',  'ГБУЗ ГП 212 ДЗМ': 'ГП 212', 'ГБУЗ ГП 19 ДЗМ': 'ГП 19', 
'ГБУЗ ГП №166 ДЗМ': 'ГП 166', 'ГБУЗ ГП 9 ДЗМ': 'ГП 9', 'ГБУЗ ГП 69 ДЗМ': 'ГП 69',
'ГБУЗ "ГП № 3 ДЗМ"': 'ГП 3', 'ГБУЗ ГП 5 ДЗМ': 'ГП 5', 'ГБУЗ ГП 46 ДЗМ': 'ГП 46', 'ГБУЗ ГП №8 ДЗМ': 'ГП 8', 
'ГБУЗ "ГП № 45 ДЗМ"': 'ГП 45'}

In [419]:
mo_list = set(rename_mo.values())

In [420]:
new_doctor_patients = \
initial_assistant_to_patient_distribution_in_today_task[(initial_assistant_to_patient_distribution_in_today_task.mo.isin(mo_list)) & 
                    (~initial_assistant_to_patient_distribution_in_today_task.doctor_fio.isin(doctors_list))].reset_index(drop=True)

In [421]:
initial_task_for_today_present_assistants = \
initial_task_for_today[~initial_task_for_today.operator_login.isin(absent_assistants.login)]

In [422]:
initial_task_for_today[initial_task_for_today.operator_login.isin(absent_assistants.login)].shape[0]

3329

In [423]:
initial_task_for_today_present_assistants.shape[0]

5885

In [424]:
initial_task_for_today_present_assistants.groupby('operator_login').agg({'emiasid': 'count'}).sort_values('emiasid', ascending=False)

,emiasid
operator_login,
pomoshnik_pdn83,295
pomoshnik_pdn52,285
pomoshnik_pdn307,249
pomoshnik_pdn65,225
pomoshnik_pdn11,200
...,...
pomoshnik_pdn57,2
pomoshnik_pdn98,2
pomoshnik_pdn264,1


In [425]:
initial_task_for_today_absent_assistants =  \
initial_task_for_today[initial_task_for_today.operator_login.isin(absent_assistants.login)] \
[['emiasid', 'status', 'old_comment', 'mo', 'priority', 'month']]

In [426]:
additional_task_for_present_assistants = \
pd.concat([
#     new_doctor_patients,
    initial_task_for_today_absent_assistants
]).reset_index(drop=True)

In [427]:
additional_task_for_present_assistants_by_mo = \
additional_task_for_present_assistants.groupby(['mo']).agg({'emiasid': 'count'}).sort_values(by=['mo', 'emiasid'])

In [428]:
distribution_count = initial_task_for_today_present_assistants.groupby(['mo']) \
.agg({'emiasid': 'count', 'operator_login': 'nunique'}) \
.sort_values(by=['mo', 'emiasid'])

In [429]:
distribution_count['total_delta'] = distribution_count['operator_login'] * task_amount - distribution_count['emiasid']

In [430]:
distribution_count = distribution_count[distribution_count.total_delta > 0]

In [431]:
additional_task_for_present_assistants

,emiasid,status,old_comment,mo,priority,month
0,18774748,Перенос звонка на другой день,"перезвонить, записать на исследования",ГП 22,2,september
1,16206328,Перенос звонка на другой день,"перезвонить, записать на исследования",ГП 22,2,september
2,2246478282,Запишется самостоятельно,"перезвонить, записать к терапевту",ГП 22,2,august
3,26788623,Перенос звонка на другой день,перезвонить,ГП 22,2,september
4,17420053,Запишется самостоятельно,"перезвонить , кп",ГП 22,2,september
...,...,...,...,...,...,...
3324,16562715,Запишется самостоятельно,"перезвонить, записать к терапевту",ГП 66,2,august
3325,173711791,Будет в Москве в другом месяце,"перезвонить, записать к офтальмологу, кровь сд...",ГП 66,2,august
3326,394621841,Перенос звонка на другой день,"перезвонить, контроль записи на исследования",ГП 66,2,september
3327,16586660,Перенос звонка на другой день,"перезвонить, записать к терапевту, нет слотов,...",ГП 66,2,september


In [432]:
final_task_predistribution = \
pd.concat([initial_task_for_today_present_assistants, additional_task_for_present_assistants]).reset_index(drop=True)

In [433]:
empty_login_patients_amount_before = final_task_predistribution[final_task_predistribution.operator_login.isna()].shape[0]
empty_login_patients_amount_before

3329

In [434]:
%%time

for mo in distribution_count.index:
    
    while final_task_predistribution[final_task_predistribution.mo == mo].groupby('operator_login') \
    .agg({'emiasid': 'count'}).min()['emiasid'] < task_amount and \
    final_task_predistribution[(final_task_predistribution.operator_login.isna()) & (final_task_predistribution.mo == mo)] \
    .shape[0] > 0:
        
        miminal_in_mo_assistant_emiasid_amount = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('operator_login').agg({'emiasid': 'count'}).min()['emiasid']

        minimal_in_mo_emiasid_amount_login = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('operator_login').agg({'emiasid': 'count'}).sort_values(by='emiasid').index[0]
        
        
        empty_login_in_mo_index = final_task_predistribution[
            (final_task_predistribution.operator_login.isna()) & (final_task_predistribution.mo == mo)
        ].index[0]
        
        final_task_predistribution = \
        final_task_predistribution.sort_values(by=['priority', 'last_call_date', 'calls_amount'])

        final_task_predistribution.loc[empty_login_in_mo_index, 'operator_login'] = minimal_in_mo_emiasid_amount_login


CPU times: total: 41.4 s
Wall time: 41.4 s


In [435]:
empty_login_patients_amount_after = final_task_predistribution[final_task_predistribution.operator_login.isna()].shape[0]
empty_login_patients_amount_after

241

In [436]:
check_table = distribution_count.merge(additional_task_for_present_assistants_by_mo,left_index=True, right_index=True) \
[['total_delta', 'emiasid_y']]

In [437]:
check_table.loc[check_table['total_delta'] > check_table['emiasid_y'], 'real_addition'] = \
check_table.loc[check_table['total_delta'] > check_table['emiasid_y'], 'emiasid_y']

check_table.loc[check_table['total_delta'] <= check_table['emiasid_y'], 'real_addition'] = \
check_table.loc[check_table['total_delta'] <= check_table['emiasid_y'], 'total_delta']

In [438]:
real_change_of_emty_login_patients_amount = check_table.real_addition.sum()
real_change_of_emty_login_patients_amount

3088.0

In [439]:
empty_login_patients_amount_before - empty_login_patients_amount_after == real_change_of_emty_login_patients_amount

True

In [440]:
final_task_distribution = final_task_predistribution[~final_task_predistribution.operator_login.isna()] \
.copy().reset_index(drop=True)

In [441]:
final_task_distribution.shape[0]

8973

In [442]:
final_task_distribution.columns

Index(['emiasid', 'status', 'old_comment', 'operator_login', 'mo',
       'doctor_fio', 'priority', 'calls_amount', 'last_call_date', 'month'],
      dtype='object')

In [443]:
final_task_distribution.status.value_counts()

Ни разу не звонили                4638
Перенос звонка на другой день     3921
Запишется самостоятельно           284
Будет в Москве в другом месяце     100
Не дозвонились                      19
Отказ                                4
Разговор состоялся                   3
Перезвонить сегодня                  2
Живет не в Москве                    1
Госпитализирован                     1
Name: status, dtype: int64

In [444]:
final_task_distribution.shape[0]

8973

In [445]:
final_task_distribution.columns

Index(['emiasid', 'status', 'old_comment', 'operator_login', 'mo',
       'doctor_fio', 'priority', 'calls_amount', 'last_call_date', 'month'],
      dtype='object')

In [446]:
final_task_distribution.head(1)

,emiasid,status,old_comment,operator_login,mo,doctor_fio,priority,calls_amount,last_call_date,month
0,29969042,Ни разу не звонили,NaN,pomoshnik_pdn6,ГП 23,Скотникова Мария Александровна,1,NaN,NaT,september


In [447]:
second_stage_patients.head(1)

,emiasid,operator_login,status,old_comment,next_call_date,mo,doctor_fio,phone_number,calls_amount,last_call_date,last_call_result,last_talk_result,problem,month,group,operator_absence,priority
5380,18937181,pomoshnik_pdn24,Разговор состоялся,NaN,NaT,КДЦ 4,Лосев Антон Сергеевич,9.852178e+09,2.0,2022-08-22,Разговор состоялся,NaN,NaN,august,NaN,NaN,5


In [448]:
second_stage_patients = second_stage_patients[['emiasid', 'status', 'old_comment', 'operator_login', 'mo', 'priority',
       'calls_amount', 'last_call_date', 'month']]

In [449]:
tomorrow_record_patients = tomorrow_record_patients[['emiasid', 'status', 'old_comment', 'operator_login', 'mo', 'priority',
       'calls_amount', 'last_call_date', 'month']]

In [450]:
tomorrow_record_patients

,emiasid,status,old_comment,operator_login,mo,priority,calls_amount,last_call_date,month


In [451]:
final_actual_patients_for_task_ = pd.concat([
    final_task_distribution,
    august_patients,
    second_stage_patients,
    tomorrow_record_patients
]) \
.sort_values(['priority']).reset_index(drop=True)

final_actual_patients_for_task_.shape[0]

39057

In [452]:
# final_actual_patients_for_task = final_actual_patients_for_task_.drop_duplicates(subset=['phone_number'], keep='first')
# final_actual_patients_for_task_.shape[0]

In [453]:
final_actual_patients_for_task_ = final_actual_patients_for_task_.drop_duplicates(subset=['emiasid'], keep='first')
final_actual_patients_for_task_.shape[0]

39007

In [454]:
final_actual_patients_for_task_.head(3)

,emiasid,status,old_comment,operator_login,mo,doctor_fio,priority,calls_amount,last_call_date,month,next_call_date,phone_number,last_call_result,last_talk_result,problem,group,operator_absence
0,29969042,Ни разу не звонили,NaN,pomoshnik_pdn6,ГП 23,Скотникова Мария Александровна,1,NaN,NaT,september,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18454756,Ни разу не звонили,NaN,pomoshnik_pdn46,ГП 214,NaN,1,NaN,NaN,september,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1844336741,Ни разу не звонили,NaN,pomoshnik_pdn163,ГП 214,NaN,1,NaN,NaN,september,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [455]:
final_actual_patients_for_task_ = \
final_actual_patients_for_task_[['emiasid', 'status', 'old_comment', 'operator_login', 'mo', 'priority', 'calls_amount',
                                'last_call_date', 'month']]

In [456]:
final_actual_patients_for_task_['emiasid'] = final_actual_patients_for_task_['emiasid'].astype(str).str.replace('\.0', '')

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_5084\309388853.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  final_actual_patients_for_task_['emiasid'] = final_actual_patients_for_task_['emiasid'].astype(str).str.replace('\.0', '')


In [457]:
initial_assistant_to_patient_distribution_in_today_task = \
final_actual_patients_for_task_.copy()

In [458]:
initial_assistant_to_patient_distribution_in_today_task.old_comment.nunique()

2810

In [459]:
initial_assistant_to_patient_distribution_in_today_task.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(
    ['mo', 'emiasid']).head(20)

emiasid
mo     operator_login           
ГП 107 pomoshnik_pdn9          1
       pomoshnik_pdn130       52
       pomoshnik_pdn148      214
       pomoshnik_pdn43       243
       pomoshnik_pdn129      258
       pomoshnik_pdn109      272
       pomoshnik_pdn185      287
ГП 109 pomoshnik_pdn80         1
       pomoshnik_pdn95        11
       pomoshnik_pdn155       35
       pomoshnik_pdn134      188
       pomoshnik_pdn168      205
ГП 11  pomoshnik_pdn146      111
       pomoshnik_pdn32       160
       pomoshnik_pdn184      304
       pomoshnik_pdn147      358
ГП 134 pomoshnik_pdn249        1
       pomoshnik_pdn57         1
       pomoshnik_pdn93       233
       pomoshnik_pdn63       235

In [460]:
# initial_assistant_to_patient_distribution_in_today_task.to_excel('initial_assistant_to_patient_distribution_in_today_task.xlsx')

In [461]:
set(present_assistants.login.unique()) - \
(set(initial_assistant_to_patient_distribution_in_today_task.operator_login.unique()) - {np.nan})

{'pomoshnik_pdn131'}

In [462]:
task_amount = 120

In [463]:
%%time

initial_task_for_today = pd.DataFrame()
for operator_login in set(initial_assistant_to_patient_distribution_in_today_task.operator_login):
    initial_assistant_to_patient_distribution_in_today_task_for_operator_login = \
    initial_assistant_to_patient_distribution_in_today_task[initial_assistant_to_patient_distribution_in_today_task \
                                                            .operator_login
         == operator_login].sort_values(by=['priority', 'last_call_date', 'calls_amount'])
    call_back_number_for_operator_login = initial_assistant_to_patient_distribution_in_today_task_for_operator_login[
        initial_assistant_to_patient_distribution_in_today_task_for_operator_login.status.isin({
            'Перезвонить сегодня', 'Перенос звонка на другой день'})].shape[0]

    initial_task_for_today = pd.concat(
        [initial_task_for_today, 

         
         initial_assistant_to_patient_distribution_in_today_task[
             (initial_assistant_to_patient_distribution_in_today_task.operator_login==operator_login)
         ].head(task_amount),
        
       
        ]).reset_index(drop=True)

initial_task_for_today.shape[0]

CPU times: total: 2.45 s
Wall time: 2.45 s


21748

In [464]:
initial_task_for_today \
.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'])\
.sort_values(by='emiasid', ascending=True).head(10)

emiasid
mo     operator_login           
ГП 107 pomoshnik_pdn9          1
ДЦ 5   pomoshnik_pdn185        1
ГП 209 pomoshnik_pdn268        1
       pomoshnik_pdn32         1
ГП 218 pomoshnik_pdn134        1
ГП 66  pomoshnik_pdn57         1
ДКЦ 1  pomoshnik_pdn29         1
ГП 170 pomoshnik_pdn151        1
ГП 45  pomoshnik_pdn208        1
ГП 166 pomoshnik_pdn85         1

In [465]:
initial_task_for_today \
.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'])\
.sort_values(by='emiasid', ascending=False).head(10)

emiasid
mo    operator_login           
ГП 45 pomoshnik_pdn307      120
ГП 64 pomoshnik_pdn79       120
ГП 52 pomoshnik_pdn197      120
      pomoshnik_pdn220      120
ГП 62 pomoshnik_pdn106      120
      pomoshnik_pdn107      120
      pomoshnik_pdn108      120
      pomoshnik_pdn267      120
ГП 64 pomoshnik_pdn45       120
      pomoshnik_pdn47       120

In [466]:
rename_mo = \
{'ГБУЗ Г МОСКВЫ ГП № 170 ДЗМ': 'ГП 170', ' ГП 6 ДЗМ': 'ГП 6', 'ГБУЗ КДЦ 6 ДЗМ':'КДЦ 6', 
             'ГБУЗ "ГП № 134 ДЗМ"': 'ГП 134', 'КДЦ 4': 'КДЦ 4', 'ГБУЗ ДКЦ 1 ДЗМ': 'ДКЦ 1', 'ГБУЗ ГП 23 ДЗМ': 'ГП 23',
             'ГБУЗ ГП 109 ДЗМ': 'ГП 109', 'ГБУЗ ГП 11 ДЗМ': 'ГП 11', 'ГБУЗ ГП 64 ДЗМ': 'ГП 64', 'ГБУЗ ГП 191 ДЗМ': 'ГП 191', 
             'ГБУЗ ГП 220 ДЗМ': 'ГП 220', 'ГБУЗ "ГП № 218 ДЗМ"': 'ГП 218',  'ГБУЗ ГП 107 ДЗМ': 'ГП 107', 
             'ГБУЗ "ГП № 22 ДЗМ"': 'ГП 22', 'ГБУЗ ДЦ 5 ДЗМ': 'ДЦ 5', 'ГБУЗ ГП 210 ДЗМ': 'ГП 210', 'ГБУЗ ГП 214 ДЗМ': 'ГП 214', 
             'ГБУЗ ГП 2 ДЗМ': 'ГП 2', 'ГБУЗ КДП 121 ДЗМ': 'КДП 121', 'ГБУЗ ГП 66 ДЗМ': 'ГП 66',  'ГБУЗ ДЦ 3 ДЗМ ': 'ДЦ 3',
             'ГБУЗ ГП 36 ДЗМ': 'ГП 36',  'ГБУЗ ГП 62 ДЗМ ': 'ГП 62', 
            'ГБУЗ "ГП № 52 ДЗМ"': 'ГП 52', 'ГБУЗ "ГП 67 ДЗМ"': 'ГП 67', 'ГБУЗ ГП 68 ДЗМ': 'ГП 68', 'ГБУЗ ГП 175 ДЗМ': 'ГП 175',
'ГБУЗ ГП № 209 ДЗМ': 'ГП 209',  'ГБУЗ ГП 212 ДЗМ': 'ГП 212', 'ГБУЗ ГП 19 ДЗМ': 'ГП 19', 
'ГБУЗ ГП №166 ДЗМ': 'ГП 166', 'ГБУЗ ГП 9 ДЗМ': 'ГП 9', 'ГБУЗ ГП 69 ДЗМ': 'ГП 69',
'ГБУЗ "ГП № 3 ДЗМ"': 'ГП 3', 'ГБУЗ ГП 5 ДЗМ': 'ГП 5', 'ГБУЗ ГП 46 ДЗМ': 'ГП 46', 'ГБУЗ ГП №8 ДЗМ': 'ГП 8', 
'ГБУЗ "ГП № 45 ДЗМ"': 'ГП 45'}

In [467]:
mo_list = set(rename_mo.values())

In [468]:
# new_doctor_patients = \
# initial_assistant_to_patient_distribution_in_today_task[(initial_assistant_to_patient_distribution_in_today_task.mo.isin(mo_list)) & 
#                     (~initial_assistant_to_patient_distribution_in_today_task.doctor_fio.isin(doctors_list))].reset_index(drop=True)
# new_doctor_patients = new_doctor_patients[~new_doctor_patients.emiasid.isin(calls_data.emiasid)]

In [469]:
initial_task_for_today_present_assistants = \
initial_task_for_today[~initial_task_for_today.operator_login.isin(absent_assistants.login)]

In [470]:
initial_task_for_today[initial_task_for_today.operator_login.isin(absent_assistants.login)].shape[0]

3988

In [471]:
initial_task_for_today_present_assistants.shape[0]

17760

In [472]:
initial_task_for_today_present_assistants.groupby('operator_login').agg({'emiasid': 'count'}).sort_values('emiasid', ascending=False)

,emiasid
operator_login,
pomoshnik_pdn100,120
pomoshnik_pdn39,120
pomoshnik_pdn309,120
pomoshnik_pdn31,120
pomoshnik_pdn32,120
...,...
pomoshnik_pdn242,16
pomoshnik_pdn240,13
pomoshnik_pdn200,9


In [473]:
initial_task_for_today_absent_assistants =  \
initial_task_for_today[initial_task_for_today.operator_login.isin(absent_assistants.login)] \
[['emiasid', 'status', 'old_comment', 'mo', 'priority', 'month']]

In [474]:
additional_task_for_present_assistants = \
pd.concat([
#     new_doctor_patients,
    initial_task_for_today_absent_assistants
]).reset_index(drop=True)

In [475]:
additional_task_for_present_assistants_by_mo = \
additional_task_for_present_assistants.groupby(['mo']).agg({'emiasid': 'count'}).sort_values(by=['mo', 'emiasid'])

In [476]:
distribution_count = initial_task_for_today_present_assistants.groupby(['mo']) \
.agg({'emiasid': 'count', 'operator_login': 'nunique'}) \
.sort_values(by=['mo', 'emiasid'])

In [477]:
distribution_count['total_delta'] = distribution_count['operator_login'] * task_amount - distribution_count['emiasid']

In [478]:
distribution_count = distribution_count[distribution_count.total_delta > 0]

In [479]:
additional_task_for_present_assistants

,emiasid,status,old_comment,mo,priority,month
0,19994950,Не дозвонились,NaN,ГП 52,5,september
1,19223589,Не дозвонились,NaN,ГП 52,5,september
2,2241234545,Не дозвонились,NaN,ГП 52,5,september
3,18475127,Не дозвонились,NaN,ГП 52,5,september
4,2239931529,Не дозвонились,NaN,ГП 52,5,september
...,...,...,...,...,...,...
3983,2246680300,Разговор состоялся,NaN,ГП 66,5,august
3984,28444862,Разговор состоялся,"всё прошла, ДН не закрыто",ГП 66,5,august
3985,18685932,Не дозвонились,NaN,ГП 66,5,september
3986,23962465,Не дозвонились,NaN,ГП 66,5,september


In [480]:
final_task_predistribution = \
pd.concat([initial_task_for_today_present_assistants, additional_task_for_present_assistants]).reset_index(drop=True)

In [481]:
empty_login_patients_amount_before = final_task_predistribution[final_task_predistribution.operator_login.isna()].shape[0]
empty_login_patients_amount_before

3988

In [482]:
%%time

for mo in distribution_count.index:
    
    while final_task_predistribution[final_task_predistribution.mo == mo].groupby('operator_login') \
    .agg({'emiasid': 'count'}).min()['emiasid'] < task_amount and \
    final_task_predistribution[(final_task_predistribution.operator_login.isna()) & (final_task_predistribution.mo == mo)] \
    .shape[0] > 0:
        
        miminal_in_mo_assistant_emiasid_amount = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('operator_login').agg({'emiasid': 'count'}).min()['emiasid']

        minimal_in_mo_emiasid_amount_login = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('operator_login').agg({'emiasid': 'count'}).sort_values(by='emiasid').index[0]
        
        
        empty_login_in_mo_index = final_task_predistribution[
            (final_task_predistribution.operator_login.isna()) & (final_task_predistribution.mo == mo)
        ].index[0]
        
        final_task_predistribution = \
        final_task_predistribution.sort_values(by=['priority', 'last_call_date', 'calls_amount'])

        final_task_predistribution.loc[empty_login_in_mo_index, 'operator_login'] = minimal_in_mo_emiasid_amount_login


CPU times: total: 29 s
Wall time: 29.1 s


In [483]:
empty_login_patients_amount_after = final_task_predistribution[final_task_predistribution.operator_login.isna()].shape[0]
empty_login_patients_amount_after

2618

In [484]:
final_task_distribution = final_task_predistribution[~final_task_predistribution.operator_login.isna()] \
.copy().reset_index(drop=True)

In [485]:
final_task_distribution.status.value_counts()

Не дозвонились                    10415
Перенос звонка на другой день      3612
Ни разу не звонили                 2875
Разговор состоялся                 1412
Будет в Москве в другом месяце      508
Запишется самостоятельно            249
Не нравится врач                     26
Не нравится МО                       10
Отсутствует номер                     8
Живет не в Москве                     7
Отказ                                 4
Госпитализирован                      2
Перезвонить сегодня                   2
Name: status, dtype: int64

In [248]:
final_task_distribution.operator_login.value_counts()

pomoshnik_pdn168    120
pomoshnik_pdn129    120
pomoshnik_pdn27     120
pomoshnik_pdn85     120
pomoshnik_pdn9      120
                   ... 
pomoshnik_pdn265     20
pomoshnik_pdn173     16
pomoshnik_pdn200     13
pomoshnik_pdn131      3
pomoshnik_pdn151      1
Name: operator_login, Length: 168, dtype: int64

In [249]:
final_task_distribution = final_task_distribution.rename(columns={'login': 'operator_login'})

In [250]:
final_task_distribution = final_task_distribution.rename(columns={'last_comment': 'old_comment'})

In [251]:
final_task_distribution.shape[0]

18434

In [261]:
final_task_distribution.groupby('operator_login').agg({'emiasid': 'count'}).sort_values(by=['emiasid']).head(35).emiasid.sum()

2474

In [253]:
final_task_distribution['emiasid'] = final_task_distribution['emiasid']

In [254]:
final_task_distribution = final_task_distribution.drop_duplicates(subset=['emiasid'], keep='first')
final_task_distribution.shape[0]

18434

In [255]:
final_task_distribution.status.value_counts()

Не дозвонились                    9110
Перенос звонка на другой день     4650
Ни разу не звонили                2199
Разговор состоялся                1315
Будет в Москве в другом месяце     380
Перезвонить сегодня                367
Запишется самостоятельно           361
Не нравится врач                    17
Отсутствует номер                    9
Отказ                                8
Не нравится МО                       8
Госпитализирован                     5
Живет не в Москве                    4
Пациент отрицает диагноз             1
Name: status, dtype: int64

In [256]:
final_task_distribution.last_call_date.value_counts()

2022-09-07    4711
2022-09-06    1987
2022-09-02    1868
2022-09-08    1809
2022-09-05    1207
2022-09-12     890
2022-09-09     875
2022-09-15     579
2022-09-13     328
2022-09-14     312
2022-08-16      33
2022-09-01      26
2022-08-15      21
2022-08-31      20
2022-08-12      16
2022-08-09      15
2022-08-05      15
2022-08-04      15
2022-08-03      12
2022-08-17      12
2022-08-18      11
2022-08-23      11
2022-08-10      10
2022-08-11       9
2022-08-19       7
2022-08-08       5
2022-08-22       5
2022-08-24       5
2022-08-26       5
2022-08-25       3
2022-08-29       2
2022-08-30       1
Name: last_call_date, dtype: int64

In [ ]:
final_task_distribution[final_task_distribution.month == 'august']\
.to_excel(rf'august_task_for_today_{today_y_m_d}.xlsx', index=False)

In [ ]:
final_task_distribution[final_task_distribution.month == 'september']\
.to_excel(rf'september_task_for_today_{today_y_m_d}.xlsx', index=False)

In [ ]:
final_task_distribution.to_excel(rf'full_task_for_today_{today_y_m_d}.xlsx', index=False)

In [ ]:
tomorrow_record_patients[tomorrow_record_patients.emiasid.isin(final_task_distribution.emiasid)]